# Preprocess data for viz

## Data for map 

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [ ]:
plr = gpd.read_file('data/raw-data/lor/lor_planungsraeume.geojson', encoding='utf-8')

# old data: stgbxIIlor.csv
df = pd.read_csv('data/raw-data/lor/2018_new.csv', sep=";", dtype={'Kennung':str}, na_values=['.', 'x'], encoding='utf-8')
df.set_index('Kennung', inplace=True)

data = df.join(plr.set_index('spatial_name'), rsuffix='_r')

data.rename(columns={
       ' je 100 der Bevölkerung1)\n(18 - 64 Jahre,\ninsgesamt)': '18-64 Jahre insgesamt',
       ' je 100 der Bevölkerung1)\n(ab 65 Jahre,\ninsgesamt)': 'ab 65 Jahre insgesamt',
       ' je 100 der Bevölkerung1)\n(18 - 64 Jahre,\nDeutsche)': '18-64 Jahre Deutsche',
       ' je 100 der Bevölkerung1)\n(ab 65 Jahre,\nDeutsche)': 'ab 65 Jahre Deutsche',
       ' je 100 der Bevölkerung1)\n(18 - 64 Jahre,\nAusländer)': '18-64 Jahre Ausländer',
       ' je 100 der Bevölkerung1)\n(ab 65 Jahre,\nAusländer)': 'ab 65 Jahre Ausländer'
}, inplace=True)

gpd.GeoDataFrame(data).to_file('data/preprocessed-data/xii.geojson', driver='GeoJSON', encoding='utf-8')

## Merge all years for whole Berlin (linechart)

In [6]:
full_df = pd.DataFrame()

for i in range (2006, 2019):
    df = pd.read_excel('data/raw-data/lor/{}.xls'.format(str(i)), sheet_name='Tab E1', header=2, na_values=['x', '.'])
    df['jahr'] = i
    full_df = full_df.append(df[df.iloc[:,1] == 'Berlin'], sort=False)

full_df.set_index('jahr', inplace = True)
full_df = full_df.round(2)

full_df.rename(columns={
       ' je 100 der Bevölkerung1)\n(18 - 64 Jahre,\ninsgesamt)': '18-64 Jahre insgesamt',
       ' je 100 der Bevölkerung1)\n(ab 65 Jahre,\ninsgesamt)': 'ab 65 Jahre insgesamt',
       ' je 100 der Bevölkerung1)\n(18 - 64 Jahre,\nDeutsche)': '18-64 Jahre Deutsche',
       ' je 100 der Bevölkerung1)\n(ab 65 Jahre,\nDeutsche)': 'ab 65 Jahre Deutsche',
       ' je 100 der Bevölkerung1)\n(18 - 64 Jahre,\nAusländer)': '18-64 Jahre Ausländer',
       ' je 100 der Bevölkerung1)\n(ab 65 Jahre,\nAusländer)': 'ab 65 Jahre Ausländer'
}, inplace=True)

full_df[['18-64 Jahre insgesamt',
       'ab 65 Jahre insgesamt', '18-64 Jahre Deutsche', 'ab 65 Jahre Deutsche',
       '18-64 Jahre Ausländer', 'ab 65 Jahre Ausländer']].to_csv('timeline.csv', encoding='utf-8')

full_df = full_df.round(2)
full_df.to_csv('data/preprocessed-data/timeline.csv', encoding='utf-8')

## Merge all years on LOR Level (timelapse map)

In [29]:
# ab 65 Jahre

df = pd.DataFrame()

for i in range (2006, 2019):
    df_temp = pd.read_excel('data/raw-data/lor/{}.xls'.format(str(i)), sheet_name='Tab E1', header=2, na_values=['x', '.'])
    df_temp.rename(columns={' je 100 der Bevölkerung1)\n(ab 65 Jahre,\ninsgesamt)': 'y' + str(i), 
                            'Planungsraum':'Kennung', 'Unnamed: 1':'Name'}, inplace=True)

    df_temp = df_temp[df_temp.Name.notnull() & (df_temp.Name != 'Name')]
    df_temp = df_temp[['Kennung', 'Name', 'y' + str(i)]]
    
    if df.size == 0:
        df = df_temp
    else:
        df = df.merge(df_temp, how="right")

df = df.round(2)

df = df.fillna('null')

df.to_csv('data/preprocessed-data/timeline.csv', encoding='utf-8')

## Merge all years for "angerechnete Einkommen" and "Kosten der Unterkunft"

### angerechnete Einkommen

In [202]:
# this setup expects the rows to always stay in the same order (e.g.: row 6 = "Empfänger/innen insgesamt").
# If the order changes, the outcome data is faulty

df_income = pd.DataFrame()

for i in range (2007, 2019):
    df_temp = pd.read_excel('data/raw-data/monatliche-statistik/grusi_{}.xls'.format(str(i)), sheet_name='Tab E8', header=2, na_values=['x', '.'])
    df_temp.rename(columns={'Unnamed: 15': str(i), 
                            'Unnamed: 0':'nationality'}, inplace=True)
    df_temp = df_temp[df_temp.nationality.notnull() & (df_temp[str(i)].notnull())]

    
    
    if df_income.size == 0:
        df_income = df_temp[['nationality', str(i)]]
        #df.set_index('nationality', inplace = True)
    else:
        df_income[str(i)] = df_temp[str(i)]

df_income = df_income.round(2)
df_income = df_income.fillna('null')

In [203]:
df_income.loc[2,:]

nationality    Empfänger/innen insgesamt
2007                              427.31
2008                              282.02
2009                              218.07
2010                              209.07
2011                              212.26
2012                              224.22
2013                              229.71
2014                              270.73
2015                              284.14
2016                              283.75
2017                              286.04
2018                              276.57
Name: 2, dtype: object

In [204]:
df_income.to_csv('data/preprocessed-data/income.csv', encoding='utf-8')

In [205]:
# this setup expects the rows to always stay in the same order (e.g.: row 6 = "Empfänger/innen insgesamt").
# If the order changes, the outcome data is faulty

df_rent = pd.DataFrame()

for i in range (2007, 2019):
    df_temp = pd.read_excel('data/raw-data/monatliche-statistik/grusi_{}.xls'.format(str(i)), sheet_name='Tab E6', header=3, na_values=['x', '.'])
    df_temp.rename(columns={'Durchschnittliche\nanerkannte\nAufwendungen für\nUnterkunft und\nHeizung in \nEUR pro Monat\n(Spalte 4-17)': str(i), 
                            'Unnamed: 0':'nationality'}, inplace=True)
    df_temp = df_temp[df_temp.nationality.notnull() & (df_temp[str(i)].notnull())]

    
    
    if df_rent.size == 0:
        df_rent = df_temp[['nationality', str(i)]]
        #df.set_index('nationality', inplace = True)
    else:
        df_rent[str(i)] = df_temp[str(i)]

df_rent = df_rent.round(2)
df_rent = df_rent.fillna('null')

In [207]:
df_rent.to_csv('data/preprocessed-data/rent.csv', encoding='utf-8')

#### join both datasets to one

In [208]:
# get data of income dataFrame
df_joined = df_income.set_index('nationality').T.iloc[:,0:1]
df_joined.rename(columns={'Empfänger/innen insgesamt':'income'}, inplace = True)

# get data of rent dataFrame
df_joined['rent'] = df_rent.set_index('nationality').T.iloc[:,0:1]

In [209]:
df_joined.to_csv('data/preprocessed-data/rent_income.csv', encoding='utf-8')